<a href="https://colab.research.google.com/github/Nethriya/Semmelweis-and-the-Discovery-of-Handwashing/blob/main/Another_copy_of_Fine_Tune_GLiNER_Token_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install -q gliner datasets
! pip install -qU accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 12.6 MB/s eta 0:00:00


## Install required libraries

## Import libraries

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import re
import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset
import json
import ast
import random

## load dataset

In [8]:
from datasets import load_dataset

ds = load_dataset("gretelai/synthetic_pii_finance_multilingual")

## Functions for data preprocessing

In [12]:

def tokenize_text(text):
    """Tokenize the input text into a list of tokens."""
    return re.findall(r'\w+(?:[-_]\w+)*|\S', text)

def tokenize_with_positions(text):
    """Tokenize the text and keep track of token positions in the original text."""
    tokens = tokenize_text(text)
    tokens_with_positions = []
    current_pos = 0

    for token in tokens:
        start_pos = text.find(token, current_pos)
        end_pos = start_pos + len(token)
        tokens_with_positions.append({"token": token, "start": start_pos, "end": end_pos})
        current_pos = end_pos

    return tokens, tokens_with_positions

def map_entities_to_tokens(example):
    """Map entity positions to tokenized text spans."""
    text, entities = example['generated_text'], ast.literal_eval(example['pii_spans'])
    tokens, tokens_with_positions = tokenize_with_positions(text)
    spans = []

    for entity in entities:
        entity_start, entity_end, label = entity['start'], entity['end'], entity['label']
        token_start_idx = token_end_idx = None

        for i, token_info in enumerate(tokens_with_positions):
            if token_info['start'] == entity_start:
                token_start_idx = i
            if token_info['end'] == entity_end:
                token_end_idx = i

        if token_start_idx is not None and token_end_idx is not None:
            spans.append((token_start_idx, token_end_idx, label))
    example['tokenized_text'] = tokens
    example['ner'] = str(spans)

    return example

## See training data

In [13]:
example = ds['train'][0]

In [14]:
text = example['generated_text']
tokens = tokenize_text(text)
entities = ast.literal_eval(example['pii_spans'])
example = map_entities_to_tokens(example)
spans = ast.literal_eval(example['ner'])

In [15]:
for i, k in zip(entities, spans):
    print(text[i['start']:i['end']])

1st day of March, 2021
Cameron-Mcknight
81685 Lopez Lodge, Apt. 6502
Cameron-Mcknight
Jann N. Butte
81685 Lopez Lodge, Apt. 6502
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
one (1) year


In [16]:
for i in spans:
    print(" ".join(tokens[i[0]:i[1]+1]))

1st day of March , 2021
Cameron-Mcknight
81685 Lopez Lodge , Apt . 6502
Cameron-Mcknight
Jann N . Butte
81685 Lopez Lodge , Apt . 6502
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
Cameron-Mcknight
one ( 1 ) year


In [17]:
for i, k in zip(ast.literal_eval(example['pii_spans']), spans):
    if text[i['start']:i['end']].replace(" ", "") != "".join(tokens[k[0]:k[1]+1]):
        print(text[i['start']:i['end']], " ".join(tokens[k[0]:k[1]+1]))

## Clean and transform data

In [18]:
filtered_ds = ds.filter(lambda x: len(x['pii_spans']) > 3).remove_columns([
        'level_0', 'document_type', 'document_description', 'expanded_type',
        'expanded_description', 'language', 'language_description', 'domain',
        'conformance_score', 'quality_score', 'toxicity_score', 'bias_score',
        'groundedness_score'
    ])

Filter:   0%|          | 0/50346 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5594 [00:00<?, ? examples/s]

In [19]:
filtered_ds = filtered_ds.map(map_entities_to_tokens)
filtered_ds

Map:   0%|          | 0/46890 [00:00<?, ? examples/s]

Map:   0%|          | 0/5240 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'generated_text', 'pii_spans', 'tokenized_text', 'ner'],
        num_rows: 46890
    })
    test: Dataset({
        features: ['index', 'generated_text', 'pii_spans', 'tokenized_text', 'ner'],
        num_rows: 5240
    })
})

In [20]:
final_df = filtered_ds.filter(lambda x: len(x['ner']) > 3).remove_columns(['index', 'generated_text', 'pii_spans'])

Filter:   0%|          | 0/46890 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5240 [00:00<?, ? examples/s]

In [21]:
# Combine the lists from both 'train'
train_dataset = []

# Extract and extend the combined list from the 'train' split
for example in final_df['train']:
    train_dataset.extend([{'tokenized_text': example['tokenized_text'], 'ner': ast.literal_eval(example['ner'])}])

In [22]:
# Combine the lists from both 'train'
test_dataset = []

# Extract and extend the combined list from the 'train' split
for example in final_df['test']:
    test_dataset.extend([{'tokenized_text': example['tokenized_text'], 'ner': ast.literal_eval(example['ner'])}])

In [23]:
for i, j in test_dataset[0].items():
    print(i, j)

tokenized_text ['UNB', '+', 'UNOC', ':', '300', '+', 'IA1234567890', ':', '1234567890', '+', 'IA9876543210', ':', '9876543210', '+', '161223', ':', '1230', '+', '0001', '+', 'EA', '+', 'UNH', '+', 'SHP', '+', '20121212121212', ':', '1234567890', '+', 'SHP', ':', 'D', ':', '9B', ':', 'UN', ':', 'EAN008', ':', '2', ':', '200', '+', 'IA1234567890', '+', 'IA9876543456', 'BGM', '+', '220', '+', '345678901234567890', '+', '161223', '+', '1230', '+', '1234567890X', '+', '233', 'NAD', '+', 'BY', '+', '9876543210', '+', '123', 'Main', 'Street', '+', 'London', '+', 'EC3A', '8DS', '+', 'UK', '+', 'GB', '+', '8888888888888', 'NAD', '+', 'SU', '+', '1234567890', '+', '030', 'Campbell', 'Motorway', '+', 'London', '+', 'E14', '5JP', '+', 'UK', '+', 'GB', '+', '8888888888888', 'NAD', '+', 'DP', '+', '1234567890', '+', '3512', 'West', '12th', 'Street', '+', 'New', 'York', '+', 'NY', '+', '10011', '+', 'US', '+', '1', '+', '8888888888888', 'LIN', '+', '1', '+', '1234567890', '+', '20', '+', '1', '+', 'E

## Train

In [24]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = GLiNER.from_pretrained("urchade/gliner_small")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [25]:
model.to(device)
print("done")

done


In [26]:
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [ ]:
# calculate number of epochs
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 421 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 400 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss
500,145.227900,98.836899
1000,93.696500,81.150436
1500,79.542600,74.409218
2000,70.486100,69.898445
2500,69.994000,65.471657
3000,64.432900,64.116364


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 420 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 404 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 500 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 416 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packag

Skipping iteration due to error: CUDA out of memory. Tried to allocate 672.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 619.06 MiB is free. Process 3594 has 14.14 GiB memory in use. Of the allocated memory 13.08 GiB is allocated by PyTorch, and 946.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 4831 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 401 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 555 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 452 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWa

In [ ]:
trained_model = GLiNER.from_pretrained("models/checkpoint-5821", load_tokenizer=True)

In [ ]:
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import create_repo, Repository, login

# Log in to Hugging Face
login()

trained_model.push_to_hub("gliner-fine-tuned-pii-finance-multilingual-latest")

## Model Test

In [ ]:
example = ds['test'][5]
example

### Test with fine tuned model

In [ ]:
fine_tuned_model = GLiNER.from_pretrained("Mit1208/gliner-fine-tuned-pii-finance-multilingual")

In [ ]:
text = example['generated_text']

# Labels for entity prediction
labels = ["street_address", "company", "date_of_birth", "email", "date", "name"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = fine_tuned_model.predict_entities(text, labels, threshold=0.85)

# Display predicted entities and their labels
for entity in entities:
    print("(", entity["text"], "=>", entity["label"], ") (start & end ==>", entity["start"], "&", entity["end"], ")")

### Test with base model

In [ ]:
base_model = GLiNER.from_pretrained("urchade/gliner_small")

In [ ]:
text = example['generated_text']

# Labels for entity prediction
labels = ["street_address", "company", "date_of_birth", "email", "date", "name"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = base_model.predict_entities(text, labels, threshold=0.85)

# Display predicted entities and their labels
for entity in entities:
    print("(", entity["text"], "=>", entity["label"], ") (start & end ==>", entity["start"], "&", entity["end"], ")")

## References
1. https://github.com/urchade/GLiNER/blob/main/examples/finetune.ipynb
2. https://github.com/urchade/GLiNER/blob/main/examples/synthetic_data_generation.ipynb

## Test dataset creation process

In [ ]:
for i in filtered_ds['test']:
  if(i['index']==36927):
    text = i['generated_text']
    entities = ast.literal_eval(i['pii_spans'])
    break

In [ ]:
text = "I am MIT PATEL. MIT is one of the best University"
tokens = tokenize_text(text)
entities = ast.literal_eval("""[{'start': 5, 'end': 8, 'label': 'first_name'},
 {'start': 9, 'end': 14, 'label': 'last_name'},
 {'start': 16, 'end': 19, 'label': 'univeristy'}]""")

In [ ]:
entities

In [ ]:
tokens, spans = map_entities_to_tokens(text, entities)
spans